# Практическое задание

## Урок 6. Задача look-alike

1. взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)
3. сделать feature engineering
4. обучить любой классификатор (какой вам нравится)
5. далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть
6. применить random negative sampling для построения классификатора в новых условиях
7. сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)
8. поэкспериментировать с долей P на шаге 5 (как будет меняться качество модели при уменьшении/увеличении размера P)

In [1]:
import pandas as pd
import numpy as np

ссылка на датасет: https://www.kaggle.com/datasets/imnoob/kidney-disease

In [2]:
data = pd.read_csv('kidney_disease.csv')
data.head(3)

,id,age,bp,sg,al,su,rbc,pc,pcc,ba,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,0,48.0,80.0,1.02,1.0,0.0,NaN,normal,notpresent,notpresent,...,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,1,7.0,50.0,1.02,4.0,0.0,NaN,normal,notpresent,notpresent,...,38,6000,NaN,no,no,no,good,no,no,ckd
2,2,62.0,80.0,1.01,2.0,3.0,normal,normal,notpresent,notpresent,...,31,7500,NaN,no,yes,no,poor,no,yes,ckd


столбец id не является признаком, удалим его

In [3]:
data.drop (columns=data.columns[0], axis= 1 , inplace= True )
data.head(3)

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,48.0,80.0,1.02,1.0,0.0,NaN,normal,notpresent,notpresent,121.0,...,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,7.0,50.0,1.02,4.0,0.0,NaN,normal,notpresent,notpresent,NaN,...,38,6000,NaN,no,no,no,good,no,no,ckd
2,62.0,80.0,1.01,2.0,3.0,normal,normal,notpresent,notpresent,423.0,...,31,7500,NaN,no,yes,no,poor,no,yes,ckd


In [4]:
print(data.shape)

(400, 25)


In [5]:
data.iloc[:, -1].value_counts()

ckd       248
notckd    150
ckd\t       2
Name: classification, dtype: int64

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             391 non-null    float64
 1   bp              388 non-null    float64
 2   sg              353 non-null    float64
 3   al              354 non-null    float64
 4   su              351 non-null    float64
 5   rbc             248 non-null    object 
 6   pc              335 non-null    object 
 7   pcc             396 non-null    object 
 8   ba              396 non-null    object 
 9   bgr             356 non-null    float64
 10  bu              381 non-null    float64
 11  sc              383 non-null    float64
 12  sod             313 non-null    float64
 13  pot             312 non-null    float64
 14  hemo            348 non-null    float64
 15  pcv             330 non-null    object 
 16  wc              295 non-null    object 
 17  rc              270 non-null    obj

обработка и перевод object в int

In [7]:
for el in data['pcv']:
    try:
        el = int(el)
        if (isinstance(el, (int, float))):
            pass
        else:
            el = 0
#             print(el)
#             pass
    except:
        el = 0
#         print(el)

list_convert_feats = ['pcv', 'wc', 'rc']
    
for feat in list_convert_feats:
    data[feat] = pd.to_numeric(data[feat], errors='coerce')
    data = data.dropna(subset=[feat])
    for el in list_convert_feats[:2]:                                 
        data[feat].astype('int')
    for el in list_convert_feats[2]:                                 
        data[feat].astype('int')                      

переводим target в бинарный вид

In [8]:
data['classification'] = data['classification'].apply(lambda x: 1 if x == 'ckd' or x == 'ckd\t' else 0)

перевод категориальных признаков в бинарные

In [9]:
list_feats_to_bin = ['rbc', 'pc','pcc', 'ba', 'htn', 'dm', 'cad', 'pe', 'ane', 'appet'] 

def feats_to_bin(feats_to_bin):
    data[feats_to_bin] = data[feats_to_bin].apply(lambda x: 1 if (x == 'normal') or (x == 'present') or (x == 'yes') or (x == 'good') else 0)


for feat in list_feats_to_bin:
    feats_to_bin(feat)

замена пропусков в непрерывных признаках

In [10]:
list_gap_con = ['age', 'bp', 'sg', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hemo', 'pcv', 'wc', 'rc']

for gap in list_gap_con:
    data[gap] = data[gap]. fillna(data[gap].mean())

In [11]:
list_gap_bin = ['rbc', 'pc', 'pcc', 'ba', 'htn', 'dm', 'cad', 'appet', 'pe', 'ane']

for gap in list_gap_bin:
    data[gap] = data[gap].fillna(data[gap].mode())

соотношение классов целевой переменной 

In [12]:
data.iloc[:, -1].value_counts()

0    142
1    122
Name: classification, dtype: int64

In [13]:
data.head(3)

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,48.0,80.0,1.020,1.0,0.0,0,1,0,0,121.0,...,44.0,7800.0,5.2,1,1,0,1,0,0,1
3,48.0,70.0,1.005,4.0,0.0,1,0,1,0,117.0,...,32.0,6700.0,3.9,1,0,0,0,1,1,1
4,51.0,80.0,1.010,2.0,0.0,1,1,0,0,106.0,...,35.0,7300.0,4.6,0,0,0,1,0,0,1


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 264 entries, 0 to 399
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             264 non-null    float64
 1   bp              264 non-null    float64
 2   sg              264 non-null    float64
 3   al              264 non-null    float64
 4   su              264 non-null    float64
 5   rbc             264 non-null    int64  
 6   pc              264 non-null    int64  
 7   pcc             264 non-null    int64  
 8   ba              264 non-null    int64  
 9   bgr             264 non-null    float64
 10  bu              264 non-null    float64
 11  sc              264 non-null    float64
 12  sod             264 non-null    float64
 13  pot             264 non-null    float64
 14  hemo            264 non-null    float64
 15  pcv             264 non-null    float64
 16  wc              264 non-null    float64
 17  rc              264 non-null    flo

переопределим индексы

In [15]:
data = data.reset_index(drop=True)

In [16]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin

class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]


from sklearn.preprocessing import StandardScaler


# непрерывные признаки 
continuos_cols = ['age', 'bp', 'sg', 'al', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hemo', 'su', 'pcv', 'wc', 'rc'] 

# категориальные признаки 
cat_cols = ['rbc', 'pc', 'pcc', 'ba', 'htn', 'dm', 'cad', 'pe', 'ane', 'appet']

# base_cols = []


continuos_transformers = []
cat_transformers = []
# base_transformers=[]


for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
# for base_col in base_cols:
#     base_transformer = Pipeline([
#                 ('selector', NumberSelector(key=base_col))
#             ])
#     base_transformers.append((base_col, base_transformer))


разобьем выборку на тренировочную и тестовую части

In [17]:
from sklearn.model_selection import train_test_split

x_data = data.iloc[:,:-1]
y_data = data.iloc[:,-1]

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=7)

обучим модель xgboost

In [18]:
import xgboost as xgb

model = xgb.XGBClassifier()

model.fit(x_train, y_train)
y_predict = model.predict(x_test)

проверяем качество

In [19]:
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score

def evaluate_results(y_test, y_predict):
    print('Classification results:')
    f1 = f1_score(y_test, y_predict)
    print("f1: %.2f%%" % (f1 * 100.0)) 
    roc = roc_auc_score(y_test, y_predict)
    print("roc: %.2f%%" % (roc * 100.0)) 
    rec = recall_score(y_test, y_predict, average='binary')
    print("recall: %.2f%%" % (rec * 100.0)) 
    prc = precision_score(y_test, y_predict, average='binary')
    print("precision: %.2f%%" % (prc * 100.0)) 

    
evaluate_results(y_test, y_predict)

Classification results:
f1: 100.00%
roc: 100.00%
recall: 100.00%
precision: 100.00%


In [20]:
mod_data = data.copy()
#get the indices of the positives samples
pos_ind = np.where(mod_data.iloc[:,-1].values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)
# leave just 25% of the positives marked
pos_sample_len = int(np.ceil(0.25 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 31/122 as positives and unlabeling the rest


In [21]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

target variable:
 -1    233
 1     31
Name: class_test, dtype: int64


In [22]:
mod_data.head(10)

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,wc,rc,htn,dm,cad,appet,pe,ane,classification,class_test
0,48.0,80.00000,1.020,1.0,0.0,0,1,0,0,121.0,...,7800.0,5.2,1,1,0,1,0,0,1,-1
1,48.0,70.00000,1.005,4.0,0.0,1,0,1,0,117.0,...,6700.0,3.9,1,0,0,0,1,1,1,-1
2,51.0,80.00000,1.010,2.0,0.0,1,1,0,0,106.0,...,7300.0,4.6,0,0,0,1,0,0,1,-1
3,60.0,90.00000,1.015,3.0,0.0,0,0,0,0,74.0,...,7800.0,4.4,1,1,0,1,1,0,1,-1
4,24.0,76.24031,1.015,2.0,4.0,1,0,0,0,410.0,...,6900.0,5.0,0,1,0,1,1,0,1,-1
5,52.0,100.00000,1.015,3.0,0.0,1,0,1,0,138.0,...,9600.0,4.0,1,1,0,1,0,1,1,-1
6,53.0,90.00000,1.020,2.0,0.0,0,0,1,0,70.0,...,12100.0,3.7,1,1,0,0,0,1,1,-1
7,63.0,70.00000,1.010,3.0,0.0,0,0,1,0,380.0,...,4500.0,3.8,1,1,0,0,1,0,1,1
8,68.0,70.00000,1.015,3.0,1.0,0,1,1,0,208.0,...,12200.0,3.4,1,1,1,0,1,0,1,1
9,68.0,80.00000,1.010,3.0,2.0,1,0,1,1,157.0,...,11000.0,2.6,1,1,1,0,1,0,1,-1


In [23]:
x_data = mod_data.iloc[:,:-2].values # just the X 
y_labeled = mod_data.iloc[:,-1].values # new class (just the P & U)
y_positive = mod_data.iloc[:,-2].values # original class

### random negative sampling  - 25 %

In [24]:
mod_data = mod_data.sample(frac=1)
neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
pos_sample = mod_data[mod_data['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(31, 26) (31, 26)


In [25]:
model = xgb.XGBClassifier()

model.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)
y_predict = model.predict(sample_test.iloc[:,:-2].values)
evaluate_results(sample_test.iloc[:,-2].values, y_predict)

Classification results:
f1: 98.70%
roc: 98.72%
recall: 97.44%
precision: 100.00%


In [26]:
metrics_pivot = {'treatment': [],'f1': [],'roc': [],'recall':[],'precision':[]}

In [27]:
metrics_pivot['treatment'].append('rfc_25%_rns')
metrics_pivot['f1'].append(f1_score(sample_test.iloc[:,-2].values, y_predict))
metrics_pivot['roc'].append(roc_auc_score(sample_test.iloc[:,-2].values, y_predict))
metrics_pivot['recall'].append(recall_score(sample_test.iloc[:,-2].values, y_predict, average='binary'))
metrics_pivot['precision'].append(precision_score(sample_test.iloc[:,-2].values, y_predict, average='binary'))

### random negative sampling  - 10 %

In [28]:
mod_data = data.copy()
#get the indices of the positives samples
pos_ind = np.where(mod_data.iloc[:,-1].values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)

pos_sample_len = int(np.ceil(0.1 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 13/122 as positives and unlabeling the rest


In [29]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

target variable:
 -1    251
 1     13
Name: class_test, dtype: int64


In [30]:
x_data = mod_data.iloc[:,:-2].values # just the X 
y_labeled = mod_data.iloc[:,-1].values # new class (just the P & U)
y_positive = mod_data.iloc[:,-2].values # original class

In [31]:
mod_data = mod_data.sample(frac=1)
neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
pos_sample = mod_data[mod_data['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(13, 26) (13, 26)


In [32]:
model = xgb.XGBClassifier()

model.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)
y_predict = model.predict(sample_test.iloc[:,:-2].values)
evaluate_results(sample_test.iloc[:,-2].values, y_predict)

Classification results:
f1: 91.35%
roc: 92.23%
recall: 90.48%
precision: 92.23%


In [33]:
metrics_pivot['treatment'].append('rfc_10%_rns')
metrics_pivot['f1'].append(f1_score(sample_test.iloc[:,-2].values, y_predict))
metrics_pivot['roc'].append(roc_auc_score(sample_test.iloc[:,-2].values, y_predict))
metrics_pivot['recall'].append(recall_score(sample_test.iloc[:,-2].values, y_predict, average='binary'))
metrics_pivot['precision'].append(precision_score(sample_test.iloc[:,-2].values, y_predict, average='binary'))

### random negative sampling  - 50 %

In [34]:
mod_data = data.copy()
#get the indices of the positives samples
pos_ind = np.where(mod_data.iloc[:,-1].values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)

pos_sample_len = int(np.ceil(0.5 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 61/122 as positives and unlabeling the rest


In [35]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

target variable:
 -1    203
 1     61
Name: class_test, dtype: int64


In [36]:
x_data = mod_data.iloc[:,:-2].values # just the X 
y_labeled = mod_data.iloc[:,-1].values # new class (just the P & U)
y_positive = mod_data.iloc[:,-2].values # original class

In [37]:
mod_data = mod_data.sample(frac=1)
neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
pos_sample = mod_data[mod_data['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(61, 26) (61, 26)


In [38]:
model = xgb.XGBClassifier()

model.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)
y_predict = model.predict(sample_test.iloc[:,:-2].values)
evaluate_results(sample_test.iloc[:,-2].values, y_predict)

Classification results:
f1: 94.62%
roc: 97.45%
recall: 100.00%
precision: 89.80%


In [39]:
metrics_pivot['treatment'].append('rfc_50%_rns')
metrics_pivot['f1'].append(f1_score(sample_test.iloc[:,-2].values, y_predict))
metrics_pivot['roc'].append(roc_auc_score(sample_test.iloc[:,-2].values, y_predict))
metrics_pivot['recall'].append(recall_score(sample_test.iloc[:,-2].values, y_predict, average='binary'))
metrics_pivot['precision'].append(precision_score(sample_test.iloc[:,-2].values, y_predict, average='binary'))

In [40]:
metrics_pivot_total = pd.DataFrame(data=metrics_pivot).sort_values(by='f1', ascending=False)
metrics_pivot_total

,treatment,f1,roc,recall,precision
0,rfc_25%_rns,0.987013,0.987179,0.974359,1.000000
2,rfc_50%_rns,0.946237,0.974490,1.000000,0.897959
1,rfc_10%_rns,0.913462,0.922306,0.904762,0.922330


Вывод: при уменьшении размера P качество модели ухудшается